In [19]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import html5lib

In [20]:
def get_source(url):
    """Return the source code of the provided url.
    
    Args: 
        url (string): URL of the page to scrape.
        
    Returns:
        response (object): HTTP response object from requests_html
    """  
    
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    
    except requests.exceptions.RequestException as e:
        print(e)

In [21]:
def scrape_google(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    response1 = get_source("https://www.google.com/search?q=" + query + '&start=10')
    
    links = list(response.html.absolute_links)
    links += list(response1.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    
    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
    
    return links

In [34]:
links = scrape_google("make worklife healthier")

In [63]:
def get_body(url):
    r1 = requests.get(url)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    coverpage_data = soup1.find_all('p')
    heading1 = soup1.find_all('h1')[0].get_text()
    heading2 = soup1.find_all('h2')
    heading2 = [heading2[i].get_text() for i in range(len(heading2))]
    data = [coverpage_data[i].get_text() for i in range(len(coverpage_data))]
    
    if len(heading1) > 0:

        headings = heading1 + " ".join(heading2)
    elif len(heading1) < 1:
        headings = " ".join(heading2)
    else:
        #Run the machine learning model here ML
        pass
    final_text = " ".join(data)
    return [headings, final_text]

In [95]:
def add_data_to_df(links):
    all_headings_text = []
    for i in links:
        curr = get_body(i)
        all_headings_text.append(curr)
        
    df = pd.DataFrame(all_headings_text, columns = ['Headings', 'Text'])
    return df

In [96]:
df = add_data_to_df(links)